# Install Dependencies

## External Dependencies

In [1]:
!pip install pandas


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


## FrostAura Dependencies

In [2]:
import os
import sys

USE_LOCAL_CODE: bool = True # NOTE: When changing this configuration, be sure to restart the interpreter in order to have sys path updates reflect.
directories_to_move_up: int = 2
current_executing_path: str = os.getcwd()
current_executing_path = current_executing_path.replace('\\', '/')
root_path: str = '/'.join(current_executing_path.split('/')[:-directories_to_move_up])

# Clean up.
%pip uninstall frostaura -y
sys.path = [p for p in sys.path if p != root_path]

if USE_LOCAL_CODE:
    sys.path.append(root_path)
else:
    %pip install -U --no-cache-dir frostaura

Note: you may need to restart the kernel to use updated packages.


# Import Dependencies

## Logging

In [3]:
import logging

logging.getLogger().disabled = True

## Imports

In [4]:
from frostaura.engines import IAssetProjectionEngine
from frostaura.engines import SimpleAssetProjectionEngine
import pandas as pd
from frostaura.engines import IAssetValuationEngine
from frostaura.engines import FinvizAssetValuationEngine
from frostaura.engines import IAssetCalculationsEngine
from frostaura.engines import SimpleAssetCalculationsEngine
from frostaura.data_access import EasyEquitiesPersonalAssetDataAccess
from frostaura.data_access import IResourcesDataAccess
from frostaura.data_access import EmbeddedResourcesDataAccess
from frostaura.data_access import HtmlResourcesDataAccess
from frostaura.data_access import IPersonalAssetDataAccess
from frostaura.data_access import IPublicAssetDataAccess
from frostaura.data_access import YahooFinanceDataAccess
from frostaura.models import ValuationResult

# Usage

## Setup

In [5]:
html_data_access: IResourcesDataAccess = HtmlResourcesDataAccess()
resource_data_access: IResourcesDataAccess = EmbeddedResourcesDataAccess()
asset_valuation_engine: IAssetValuationEngine = FinvizAssetValuationEngine(html_data_access=html_data_access)
public_asset_data_access: IPublicAssetDataAccess = YahooFinanceDataAccess()
asset_calculations_engine: IAssetCalculationsEngine = SimpleAssetCalculationsEngine(public_asset_data_access=public_asset_data_access)
personal_asset_data_access: IPersonalAssetDataAccess = EasyEquitiesPersonalAssetDataAccess(resource_data_access=resource_data_access,
                                                                         username=None,
                                                                         password=None)
instance: IAssetProjectionEngine = SimpleAssetProjectionEngine(asset_valuation_engine=asset_valuation_engine,
                                                               public_asset_data_access=public_asset_data_access)

## Project for a Single Symbol

In [6]:
n_months: int = 12*30 # 39 year
usd_zar_exchange_rate: float = 16.17
symbol: str = 'AAPL'
holdings: pd.DataFrame = personal_asset_data_access.get_personal_transactions()
holdings_with_profits: pd.DataFrame = asset_calculations_engine.calculate_holdings_profits(holdings=holdings)
initial_value: float = holdings_with_profits.loc[holdings_with_profits['symbol'] == 'AAPL']['total_current_usd'].sum()
monthly_deposit: float = 1000 / usd_zar_exchange_rate / 4

growth: pd.DataFrame = instance.project_monthly_asset_growth(n_months=n_months,
                                                             symbol=symbol,
                                                             principal_value=initial_value,
                                                             monthly_deposit=monthly_deposit)

growth

,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
month,,,,,
0,0.00000,0.000000,0.000000,0.000000,74.697050
1,15.46073,0.611893,15.460730,0.611893,90.769674
2,15.46073,0.743555,30.921459,1.355448,106.973958
3,15.46073,0.876295,46.382189,2.231743,123.310983
4,15.46073,1.010122,61.842919,3.241866,139.781835
...,...,...,...,...,...
356,15.46073,275.536241,5504.019790,28348.443831,33927.160672
357,15.46073,277.919991,5519.480519,28626.363823,34220.541393
358,15.46073,280.323268,5534.941249,28906.687091,34516.325390


## Project for a Entire Holdings (Multiple Symbols)

In [7]:
# 30 years
n_months: int = 12*30
usd_zar_exchange_rate: float = 16.17
holdings: pd.DataFrame = personal_asset_data_access.get_personal_transactions()
holdings_with_profits: pd.DataFrame = asset_calculations_engine.calculate_holdings_profits(holdings=holdings)
# Simply R 1000 converted to USD and split evenly among how many assets are held.
monthly_deposits: list = [(1000 / usd_zar_exchange_rate / len(holdings_with_profits)) for h in holdings_with_profits]
holdings_growth: pd.DataFrame = instance.project_monthly_holdings_growth(n_months=n_months,
                                                                         holdings_with_profits=holdings_with_profits,
                                                                         monthly_deposits=monthly_deposits)

holdings_growth

,month,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
0,1,61.842919,0.023089,61.842919,6.103659e+00,3.261989e+02
1,2,61.842919,0.022559,123.685838,1.363233e+01,3.955705e+02
2,3,61.842919,0.022247,185.528757,2.263265e+01,4.664137e+02
3,4,61.842919,0.022058,247.371676,3.315302e+01,5.387770e+02
4,5,61.842919,0.021948,309.214595,4.524369e+01,6.127106e+02
...,...,...,...,...,...,...
355,356,61.842919,0.041393,22016.079159,1.480936e+09,1.480958e+09
356,357,61.842919,0.041394,22077.922078,1.544886e+09,1.544909e+09
357,358,61.842919,0.041395,22139.764997,1.611600e+09,1.611622e+09
358,359,61.842919,0.041396,22201.607916,1.681196e+09,1.681218e+09


In [8]:
over_1b = holdings_growth.loc[holdings_growth['balance'] > 1000000000]

print(f'{round(over_1b.index[0] / 12, 2)} years to ${over_1b.iloc[0]["balance"]:.2f}.')

28.83 years to $1012334753.21.


In [9]:
data = {
    'Target Balance (USD)': list(),
    'ETA (Months)': list(),
    'ETA (Years)': list()
}

# $ 100 000
hundred_thousand: int = holdings_growth.loc[holdings_growth['balance'] > 100*1000]

data['Target Balance (USD)'].append(f'{hundred_thousand.iloc[0]["balance"]:.2f}')
data['ETA (Months)'].append(hundred_thousand.index[0])
data['ETA (Years)'].append(hundred_thousand.index[0] / 12)

# 100 000 0
million: int = holdings_growth.loc[holdings_growth['balance'] > 100*1000*10]

data['Target Balance (USD)'].append(f'{million.iloc[0]["balance"]:.2f}')
data['ETA (Months)'].append(million.index[0])
data['ETA (Years)'].append(million.index[0] / 12)

# 100 000 00
ten_million: int = holdings_growth.loc[holdings_growth['balance'] > 100*1000*10*10]

data['Target Balance (USD)'].append(f'{ten_million.iloc[0]["balance"]:.2f}')
data['ETA (Months)'].append(ten_million.index[0])
data['ETA (Years)'].append(ten_million.index[0] / 12)

# 100 000 00
hundred_million: int = holdings_growth.loc[holdings_growth['balance'] > 100*1000*10*100]

data['Target Balance (USD)'].append(f'{hundred_million.iloc[0]["balance"]:.2f}')
data['ETA (Months)'].append(hundred_million.index[0])
data['ETA (Years)'].append(hundred_million.index[0] / 12)

# 100 000 000
billion: int = holdings_growth.loc[holdings_growth['balance'] > 100*1000*10*1000]

data['Target Balance (USD)'].append(f'{billion.iloc[0]["balance"]:.2f}')
data['ETA (Months)'].append(billion.index[0])
data['ETA (Years)'].append(billion.index[0] / 12)

pd.DataFrame(data)

,Target Balance (USD),ETA (Months),ETA (Years)
0,100594.97,123,10.250000
1,1008328.51,181,15.083333
2,10269018.57,237,19.750000
3,103570753.47,292,24.333333
4,1012334753.21,346,28.833333
